# Short Circuit Example

This notebook contains an example of a short circuit calculation using `power-grid-model`.
The following points are covered
 * Construction of the model
 * Run short circuit calculation, and its relevant function arguments

This notebook skips most of the common features across all calculations of power-grid-model like updating model, batch calculations and error handling.

It serves as an example of how to use the Python API. For detailed API documentation, refer to
[Python API reference](https://power-grid-model.readthedocs.io/en/stable/api_reference/python-api-reference.html)
and [Native Data Interface](https://power-grid-model.readthedocs.io/en/stable/advanced_documentation/native-data-interface.html).


## Example Network

We use a simple network with 3 nodes, 1 source, 3 lines, and 2 loads. As shown below:

```
 -----------------------line_8---------------
 |                                          |
node_1 ---line_3--- node_2 ----line_5---- node_6 ---- fault_11
 |                    |                     |
source_10          sym_load_4           sym_load_7
```

The 3 nodes are connected in a triangular way by 3 lines.

In [1]:
# some basic imports
import numpy as np
import pandas as pd

from power_grid_model import LoadGenType
from power_grid_model import PowerGridModel, CalculationMethod, CalculationType, FaultType, FaultPhase
from power_grid_model import initialize_array

## Input Dataset

Please refer to [Components](https://power-grid-model.readthedocs.io/en/stable/user_manual/components.html) for detailed explanation of all component types and their input/output attributes.

In [3]:
# node
node = initialize_array("input", "node", 3)
node["id"] = np.array([1, 2, 6])
node["u_rated"] = [10.5e3, 10.5e3, 10.5e3]

# line
line = initialize_array("input", "line", 3)
line["id"] = [3, 5, 8]
line["from_node"] = [1, 2, 1]
line["to_node"] = [2, 6, 6]
line["from_status"] = [1, 1, 1]
line["to_status"] = [1, 1, 1]
line["r1"] = [0.25, 0.25, 0.25]
line["x1"] = [0.2, 0.2, 0.2]
line["c1"] = [10e-6, 10e-6, 10e-6]
line["tan1"] = [0.0, 0.0, 0.0]
line["i_n"] = [1000, 1000, 1000]

# load
sym_load = initialize_array("input", "sym_load", 2)
sym_load["id"] = [4, 7]
sym_load["node"] = [2, 6]
sym_load["status"] = [1, 1]
sym_load["type"] = [LoadGenType.const_power, LoadGenType.const_power]
sym_load["p_specified"] = [20e6, 10e6]
sym_load["q_specified"] = [5e6, 2e6]

# source
source = initialize_array("input", "source", 1)
source["id"] = [10]
source["node"] = [1]
source["status"] = [1]
source["u_ref"] = [1.0]

# source
fault = initialize_array("input", "fault", 1)
fault["id"] = [11]
fault["status"] = [1]
fault["fault_object"] = [6]
fault["fault_type"] = [FaultType.three_phase]
fault["fault_phase"] = [FaultPhase.abc]
fault["r_f"] = [0.1]
fault["x_f"] = [0.1]

# all
input_data = {
    "node": node,
    "line": line,
    "sym_load": sym_load,
    "source": source,
    "fault": fault
}


**Note: During a single calculation, all types of faults should be similar.**

## Validation (optional)
For efficiency reasons, most of the data is not explicitly validated in the power grid model. However, in most cases, a power flow calculation will fail/crash if the data is invalid. Often with a low level error message that is hard to relate to the original objects. Therfore, it is recommended to always validate your data before constructing a PowerGridModel instance.

The simplest and most effective way to validate your data is by using `assert_valid_input_data()` which will throw an error if it encounters any invalid data. See [Validation Examples](https://github.com/PowerGridModel/power-grid-model/blob/main/docs/examples/Validation%20Examples.ipynb) for more detailed information on the validation functions.

In [4]:
from power_grid_model.validation import assert_valid_input_data
assert_valid_input_data(input_data=input_data, calculation_type=CalculationType.short_circuit)

## Construction

The construction of the model is just calling the constructor of `PowerGridModel`.


In [5]:
model = PowerGridModel(input_data)

## One-time Short circuit Calculation

You can call the method `calculate_short_circuit` to do a one-time calculation based on the current network data in the model.

The short circuit calculation has the following settings as arguments along with its defaults:
 * calculation_method: CalculationMethod.iec60909,
 * voltage_scaling_factor_c: 1.1

Currently, there is only one calculation method for short-circuit which calculates as per IEC 60909. The `voltage_scaling_factor_c` is scaling of voltage source based on operational conditions.


In [ ]:
output_data = model.calculate_short_circuit(
    voltage_scaling_factor_c=1.1,
    calculation_method=CalculationMethod.iec60909)

### Result Dataset

We can also print a result dataset of node and line by converting the array to dataframe.

In [ ]:
print("------fault result------")
print(pd.DataFrame(output_data["fault"]))
print("------node result------")
print(pd.DataFrame(output_data["node"]))
print("------line result------")
print(pd.DataFrame(output_data["line"]))

## Batch Calculations

The batch calculations are mentioned in detail in the [Power Flow Example](https://github.com/PowerGridModel/power-grid-model/tree/main/docs/examples/Power%20Flow%20Example.ipynb). Short circuit batch calculations are carried out in similar way.